In [1]:
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Lambda
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

In [2]:
def create_input():
    X = []
    y = []
    df = pd.read_csv("classification_data/explore_img/explore_data.csv")
    shuffled_df = df.sample(len(df))

    print(shuffled_df.head)
    s = 0
    for entry in shuffled_df.values:
        s += 1
        # print(s)
        entry_img = cv2.imread(f"classification_data/explore_img/{entry[0]}")

        prepared_y = np.delete(entry,0)

        X.append(entry_img)
        y.append(prepared_y)

    
    X = np.asarray(X)
    y = np.asarray(y).astype(np.float32)
    return X, y


In [3]:
def compile_model():
    image_width, image_height, channels = 640, 360, 3  # Assuming RGB images

    model = Sequential()
    model.add(Input(shape=(image_height, image_width, channels)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(2, activation='linear'))

    model.compile(loss='mean_squared_error', optimizer=Adam())

    return model

model = compile_model()

In [4]:
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy'])

In [5]:
X, y = create_input()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)

<bound method NDFrame.head of           filename    x    y
35  explore_35.jpg  467   61
31  explore_31.jpg  291   50
7    explore_7.jpg  322  311
21  explore_21.jpg   40   65
13  explore_13.jpg  328   23
..             ...  ...  ...
43  explore_43.jpg  432   56
1    explore_1.jpg  437   50
59  explore_59.jpg  475   52
16  explore_16.jpg  114  159
52  explore_52.jpg  391  302

[64 rows x 3 columns]>
(51, 360, 640, 3) (51, 2)


In [6]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))


Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.1097 - loss: 4227332.5000 - val_accuracy: 0.0769 - val_loss: 4096497.2500
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.1201 - loss: 3536922.7500 - val_accuracy: 0.0769 - val_loss: 856650.0625
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 974ms/step - accuracy: 0.1305 - loss: 667198.4375 - val_accuracy: 0.9231 - val_loss: 152762.3750
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 978ms/step - accuracy: 0.8799 - loss: 102339.1172 - val_accuracy: 0.9231 - val_loss: 18015.4590
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 977ms/step - accuracy: 0.8799 - loss: 23504.1387 - val_accuracy: 0.9231 - val_loss: 45432.3750
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 941ms/step - accuracy: 0.8930 - loss: 58219.1016 - val_accuracy: 0.9231 - val_loss: 21066.9473
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 952ms/step - accuracy: 0.9112 - loss: 21519.2559 - val_accuracy: 0.6923 - val_loss: 48302.1406
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy:

In [7]:
loss, accuracy = model.evaluate(X_val, y_val)
print('Test accuracy:', accuracy)

model.save('mouse_explore.h5')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step - accuracy: 0.9231 - loss: 13184.6035


Test accuracy: 0.9230769276618958


In [11]:
model_rap = tf.keras.models.load_model('mouse_explore_2.h5')


for n in range(5):
    x_real, y_real = int(y_val[n][0]), int(y_val[n][1])
    print(x_real, y_real)
    img = np.asarray(X_val[n])
    
    tbp = np.expand_dims(img, axis=0)
    predictions = model_rap.predict(tbp)
    x, y = int(predictions[0][0]), int(predictions[0][1]),
    print(x, y, "\n",x_real, y_real)
    cv2.circle(img, (x,y), 10, (0,0,255), -1)
    cv2.circle(img, (x_real, y_real), 10, (255,0,0), -1)
    cv2.imshow('ss', img)
    cv2.waitKey(0) & 0xFF
cv2.destroyWindow('ss')

332 72
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
365 161 
 332 72
487 42
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
371 97 
 487 42
467 61
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
352 98 
 467 61
476 290
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
335 89 
 476 290
550 249
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
368 118 
 550 249


In [10]:
model_rap = tf.keras.models.load_model('mouse_explore.h5')
model_rap.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy'])

model_rap.fit(X_train, y_train, epochs=40, validation_data=(X_val, y_val))
model_rap.save('mouse_explore_2.h5')

Epoch 1/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.7073 - loss: 348292.5625 - val_accuracy: 0.9231 - val_loss: 1103485.5000
Epoch 2/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.8903 - loss: 800727.3750 - val_accuracy: 0.3846 - val_loss: 30010.0957
Epoch 3/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.4359 - loss: 80281.7266 - val_accuracy: 0.0769 - val_loss: 206005.1875
Epoch 4/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.1097 - loss: 220688.2656 - val_accuracy: 0.9231 - val_loss: 77908.8516
Epoch 5/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.8930 - loss: 87006.6484 - val_accuracy: 0.9231 - val_loss: 23671.4238
Epoch 6/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 990ms/step - accuracy: 0.8903 - loss: 26873.2695 - val_accuracy: 0.9231 - val_loss: 16648.3047
Epoch 7/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.8695 - loss: 15295.3320 - val_accuracy: 0.9231 - val_loss: 17434.6543
Epoch 8/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.8695 - loss: 